In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (16901 Collins Ave #5503, Sunny Isles Beach, F...
2     (17749 Collins Ave APT 1102, Sunny Isles Beach...
3     (881 Ocean Dr #5a, Key Biscayne, FL 33149, USA...
4     (6365 Collins Ave APT 1201, Miami Beach, FL 33...
5     (18911 Collins Ave APT 3207, Sunny Isles Beach...
6     (90 Alton Rd, Miami Beach, FL 33139, USA, (25....
7     (18101 Collins Ave APT 4604, Sunny Isles Beach...
8     (881 Ocean Dr #16d, Key Biscayne, FL 33149, US...
9     (701 Crandon Blvd APT 402, Key Biscayne, FL 33...
10    (9801 Collins Ave, Bal Harbour, FL 33154, USA,...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Jade Signature\r16901 Collins Ave PH5503\rSunn...,2/4/2022,9/16/2022,223,"$15,000,000.00",14425,"$1,633.45",Matthew Dugow,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",Jade Signature 16901 Collins Ave PH5503 Sunny ...,Jade Signature,16901 Collins Ave PH5503 Sunny Isles Beach,"(16901 Collins Ave #5503, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
2,Mansions at Acqualina\r17749 Collins Ave 1102\...,7/31/2020,9/15/2022,220,"$5,500,000.00",4609,"$1,193.32",Cassio Galiza PA,Podium Realty Group INC,Marvin Anhalt,Anhalt Realty,Mansions at Acqualina 17749 Collins Ave 1102 S...,Mansions at Acqualina,17749 Collins Ave 1102 Sunny Isles Beach,"(17749 Collins Ave APT 1102, Sunny Isles Beach...","(25.9400518, -80.12056079999999, 0.0)",25.940052,-80.120561,0.0
3,Casa del Mar\r881 Ocean Dr 5A\rKey Biscayne,3/10/2022,9/12/2022,91,"$2,640,000.00",3120,$846.15,Paula Sanchez,Unique International Properties,MariaAmancia Vial-Ducaud,BHHS EWM Realty,Casa del Mar 881 Ocean Dr 5A Key Biscayne,Casa del Mar,881 Ocean Dr 5A Key Biscayne,"(881 Ocean Dr #5a, Key Biscayne, FL 33149, USA...","(25.685686, -80.15729309999999, 0.0)",25.685686,-80.157293,0.0
4,Akoya\r6365 Collins Ave 1201\rMiami Beach,9/16/2022,9/16/2022,1,"$2,500,000.00",1761,"$1,419.65",Estevam Hirschbruch,Coldwell Banker Realty,Andres Fernandez,Miami Rent-Sell Property Mngmt,Akoya 6365 Collins Ave 1201 Miami Beach,Akoya,6365 Collins Ave 1201 Miami Beach,"(6365 Collins Ave APT 1201, Miami Beach, FL 33...","(25.8460388, -80.1196447, 0.0)",25.846039,-80.119645,0.0
5,Ocean Three Condo\r18911 Collins Ave 3207\rSun...,4/18/2022,9/16/2022,99,"$2,349,500.00",2649,$886.94,Boris Tarlo,"Florida Best Realty, Inc.",Victoria Shuster,One Sotheby's Int'l Realty,Ocean Three Condo 18911 Collins Ave 3207 Sunny...,Ocean Three Condo,18911 Collins Ave 3207 Sunny Isles Beach,"(18911 Collins Ave APT 3207, Sunny Isles Beach...","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0
6,Yacht Club at Portofino\r90 Alton Rd TH-8\rMia...,7/15/2022,9/15/2022,26,"$2,300,000.00",1750,"$1,314.29",Mariela Stochetti,Douglas Elliman,Ty Forkner,One Sotheby's Int'l Realty,Yacht Club at Portofino 90 Alton Rd TH-8 Miami...,Yacht Club at Portofino,90 Alton Rd TH-8 Miami Beach,"(90 Alton Rd, Miami Beach, FL 33139, USA, (25....","(25.7699498, -80.1382789, 0.0)",25.769950,-80.138279,0.0
7,Trump Palace\r18101 Collins Ave 4604\rSunny Is...,4/28/2022,9/13/2022,132,"$2,130,000.00",1873,"$1,137.21",Ludmila Bogatov,Bogatov Realty,Maira Bautista,Fortune Intl Realty Aventura,Trump Palace 18101 Collins Ave 4604 Sunny Isle...,Trump Palace,18101 Collins Ave 4604 Sunny Isles Beach,"(18101 Collins Ave APT 4604, Sunny Isles Beach...","(25.9440252, -80.1202751, 0.0)",25.944025,-80.120275,0.0
8,Casa del Mar\r881 Ocean Dr 16D\rKey Biscayne,5/2/2022,9/13/2022,68,"$2,000,000.00",1875,"$1,066.67",Lucia Marin,One Sotheby's Int'l Realty,Giulietta Ulloa,BHHS EWM Realty,Casa del Mar 881 Ocean Dr 16D Key Biscayne,Casa del Mar,881 Ocean Dr 16D Key Biscayne,"(881 Ocean Dr #16d, Key Biscayne, FL 33149, US...","(25.685686, -80.15729309999999, 0.0)",25.685686,-80.157293,0.0
9,Resort Villa One\r701 Crandon Blvd 402\rKey Bi...,5/24/2022,9/16/2022,91,"$1,970,000.00",1830,"$1,076.50",Elena Chacon,BHHS EWM Realty,Josefina Fiorito,Great Properties International LLC,Resort Villa One 701 Crandon Blvd 402 Key Bisc...,Resort Villa One,701 Crandon Blvd 402 Key Biscayne,"(701 Crandon Blvd APT 402, Key Biscayne, FL 33...","(25.6887137, -80.1620581, 0.0)",25.688714,-80.162058,0.0
10,B0almoral\r9801Collins Ave PH7\rBal Harbour,10/7/2021,9/16/2022,299,"$1,700,000.00",1682,"$1,010.70",Boris Vertsberger,"FIP Realty Services, LLC",Joelle Oiknine,One Sotheby's Int'l Realty,B0almoral 9801Collins Ave PH7 Bal Harbour,B,0almor

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [11]:
df.at[10,'building_name']=('Balmoral')
df.at[10,'address_only']=('9801 Collins Avenue unit PH7')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('September 11th - September 17th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_091722
